In [1]:
import os
import datetime
from os import listdir
from os.path import isfile, join
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
if 'current_dir' in locals():
    print("Directory has been set before:")
else:
    current_dir = os.getcwd()[:-9] 
print(current_dir)
os.chdir(current_dir)
submission = pd.read_csv(current_dir + 'submissions/submission_v1.csv') #submission as template (version is not important)

C:\Users\Guido vd Heijden\Desktop\Customer-Loyalty\


In [7]:
# Loading files for training and testing

# Load plain train data and test data:
data = pd.read_csv('all/train.csv')
testdata = pd.read_csv('all/test.csv')

# --------------------------------------------------------------------------------------------------------------------------

# Load all features in the folder called 'features': (comment these lines out for custom set of features)
# for f in listdir(current_dir + "/features"):
#     if (f[:4] == "test"):
#         print("this is a test feature : " + f)
#         temp = pd.read_csv(current_dir + "/features/" + f)
#         testdata.insert(testdata.shape[1], f[13:-4], temp[f[13:-4]] )
#     if (f[:5] == "train"):
#         print("this is a train feature : " + f)
#         temp = pd.read_csv(current_dir + "/features/" + f)
#         data.insert(data.shape[1]-1, f[14:-4], temp[f[14:-4]] )

# --------------------------------------------------------------------------------------------------------------------------

# Load a custom set of features: (comment these lines out for all features)
data = data.drop(['feature_1'],axis=1)
testdata = testdata.drop(['feature_1'],axis=1)
data = data.drop(['feature_2'],axis=1)
testdata = testdata.drop(['feature_2'],axis=1)
feature_names = ['test_feature_fam.csv','train_feature_fam.csv','test_feature_feature_1_is_1.csv', \
                 'test_feature_feature_1_is_2.csv','test_feature_feature_1_is_3.csv','test_feature_feature_1_is_4.csv', \
                 'test_feature_feature_1_is_5.csv','train_feature_feature_1_is_1.csv','train_feature_feature_1_is_2.csv', \
                 'train_feature_feature_1_is_3.csv','train_feature_feature_1_is_4.csv','train_feature_feature_1_is_5.csv', \
                 'test_feature_avg_purchases.csv','train_feature_avg_purchases.csv','test_feature_feature_2_is_1.csv', \
                 'test_feature_feature_2_is_2.csv','test_feature_feature_2_is_3.csv','train_feature_feature_2_is_1.csv', \
                 'train_feature_feature_2_is_2.csv','train_feature_feature_2_is_3.csv']

for f in feature_names:
    if (f[:4] == "test"):
        print("this is a test feature : " + f)
        temp = pd.read_csv(current_dir + "/features/" + f)
        testdata.insert(testdata.shape[1], f[13:-4], temp[f[13:-4]] )
    if (f[:5] == "train"):
        print("this is a train feature : " + f)
        temp = pd.read_csv(current_dir + "/features/" + f)
        data.insert(data.shape[1]-1, f[14:-4], temp[f[14:-4]] )

this is a test feature : test_feature_fam.csv
this is a train feature : train_feature_fam.csv
this is a test feature : test_feature_feature_1_is_1.csv
this is a test feature : test_feature_feature_1_is_2.csv
this is a test feature : test_feature_feature_1_is_3.csv
this is a test feature : test_feature_feature_1_is_4.csv
this is a test feature : test_feature_feature_1_is_5.csv
this is a train feature : train_feature_feature_1_is_1.csv
this is a train feature : train_feature_feature_1_is_2.csv
this is a train feature : train_feature_feature_1_is_3.csv
this is a train feature : train_feature_feature_1_is_4.csv
this is a train feature : train_feature_feature_1_is_5.csv
this is a test feature : test_feature_avg_purchases.csv
this is a train feature : train_feature_avg_purchases.csv


In [8]:
data

,first_active_month,card_id,feature_2,feature_3,fam,feature_1_is_1,feature_1_is_2,feature_1_is_3,feature_1_is_4,feature_1_is_5,avg_purchases,target
0,2017-06,C_ID_92a2005557,2,1,67,0.0,0.0,0.0,0.0,0.0,-151.036324,-0.820283
1,2017-01,C_ID_3d0044924f,1,0,62,0.0,0.0,0.0,0.0,0.0,-227.560818,0.392913
2,2016-08,C_ID_d639edf6cd,2,0,57,0.0,0.0,0.0,0.0,0.0,-15.990928,0.688056
3,2017-09,C_ID_186d6a6901,3,0,70,0.0,0.0,0.0,0.0,0.0,-48.781031,0.142495
4,2017-11,C_ID_cdbd2c0db2,3,0,72,1.0,1.0,1.0,1.0,1.0,-79.653390,-0.159749
5,2016-09,C_ID_0894217f2f,2,0,58,0.0,0.0,0.0,0.0,0.0,-23.071150,0.871585
6,2016-12,C_ID_7e63323c00,2,1,61,0.0,0.0,0.0,0.0,0.0,-48.459193,0.230129
7,2017-09,C_ID_dfa21fc124,2,1,70,0.0,0.0,0.0,0.0,0.0,-15.167070,2.135850
8,2017-08,C_ID_fe0fdac8ea,1,0,69,0.0,0.0,0.0,0.0,0.0,-9.886423,-0.065406
9,2016-08,C_ID_bf62c0b49d,2,0,57,0.0,0.0,0.0,0.0,0.0,-69.690497,0.300062


## Training
Training will be done in the following cells. First a random part of the training data is assigned as training set and the leftovers will be the validation set. By making the _trainsize_ variable round(train.shape[0]/1), the entirety of the training data is used as training set, so no evaluation will be done in the section _Testing within training set_.

In [19]:
train = np.array(data)
np.random.shuffle(train)
fraction_used = 1/2     # Fraction of dataset used for training (1 = entire dataset)
trainsize = round(train.shape[0] * fraction_used)
features = train.shape[1] - 3    # Amount of features in both training and test set

# A part of the training set can be used as a test set,
# so the Root Mean Squared Error can be found
train_l = train[:trainsize,features+2]
train_x = train[:trainsize,2:features+2]

# A validation sets if Root Mean Squared Error needs to be calculated without making a submission to Kaggle
val_l = train[trainsize:,features+2]
val_x = train[trainsize:,2:features+2]

In [20]:
def CorrPrint(pandas_dataframe, numpy_array):
    corr_matrix = np.corrcoef((train[:,2:numpy_array.shape[1]].T).astype(float))
    
    print(" "*15, sep=' ', end='', flush=True)
    for i in range(numpy_array.shape[1]-2):
        print(pandas_dataframe.columns[i+2], " "*(11 - len(pandas_dataframe.columns[i+2])), sep=' ', end='', flush=True)
    print()
    for i in range(numpy_array.shape[1]-2):
        print(pandas_dataframe.columns[i+2], " "*(11 - len(pandas_dataframe.columns[i+2])), corr_matrix[i])
    return

CorrPrint(data,train)

               feature_2   feature_3   fam         feature_1_is_1 feature_1_is_2 feature_1_is_3 feature_1_is_4 feature_1_is_5 avg_purchases target      
feature_2    [ 1.          0.0609252  -0.15822695 -0.02080972 -0.02080972 -0.02080972
 -0.02080972 -0.02080972  0.0020674  -0.00624176]
feature_3    [ 6.09252024e-02  1.00000000e+00 -1.96105443e-01 -2.87277711e-01
 -2.87277711e-01 -2.87277711e-01 -2.87277711e-01 -2.87277711e-01
  8.14744101e-04 -8.12528106e-03]
fam          [-0.15822695 -0.19610544  1.          0.09985669  0.09985669  0.09985669
  0.09985669  0.09985669  0.00181396  0.05046633]
feature_1_is_1  [-0.02080972 -0.28727771  0.09985669  1.          1.          1.
  1.          1.         -0.00482263  0.00511084]
feature_1_is_2  [-0.02080972 -0.28727771  0.09985669  1.          1.          1.
  1.          1.         -0.00482263  0.00511084]
feature_1_is_3  [-0.02080972 -0.28727771  0.09985669  1.          1.          1.
  1.          1.         -0.00482263  0.00511084]
featu

In [21]:
# Different regression models
model_LR = LinearRegression().fit(train_x,train_l)

# SVM took very long and didn't give better results
# model_SVM = svm.SVR()
# model_SVM.fit(train_x,train_l)

model = model_LR # Selected model

## Testing within training set

In [22]:
def RMSerror(y,y_pred):
    if y.shape[0] != y_pred.shape[0]:
        print("Root Mean Squared Error can't be calculated if number" \
              " of predictions isn't equal to number of labels!")
        return
    
    SE = 0
    for i in range(y.shape[0]):
        SE += np.square(y[i] - y_pred[i])
    
    RMSE = np.sqrt(SE / y.shape[0])
    return RMSE

In [23]:
if val_x.shape[0] > 0:
    predictions = model.predict(val_x)
    print(RMSerror(val_l,predictions))
else:
    print("No testing can be performed within the training set" \
         ", because the entire training set is used for the model")

3.853282438830292


## Predictions for test set (creating a submission .csv file)
Make sure to use the entire training set in the the "Training" section

In [14]:
testset = np.array(testdata)
testset_x = testset[:,2:features+2]

In [15]:
submission_predictions = model.predict(testset_x)

In [16]:
submission.target = submission_predictions

In [17]:
current_time = datetime.datetime.now().strftime("_%Y-%m-%d_%H-%M-%S")
save_name = 'submission_v' + str(len(listdir(current_dir + '/submissions')) + 1) + current_time + '.csv'
submission.to_csv('submissions/' + save_name, index = False)
print('Submission was saved under the name:' + '\033[1m' + save_name + '\033[0;0m')

# [Errno 13] Permission denied: 'submissions/submission_v0-001.csv'
#        means that the file is still opened somewhere

Submission was saved under the name:submission_v5_2019-01-20_16-36-04.csv


In [18]:
# run this after previous cel to see the submission
submission

,card_id,target
0,C_ID_0ab67a22ab,-0.325383
1,C_ID_130fd0cbdd,-0.401763
2,C_ID_b709037bc5,-0.261523
3,C_ID_d27d835a9f,-0.195241
4,C_ID_2b5e3df5c2,-0.669260
5,C_ID_5814b4f13c,-0.281925
6,C_ID_a1b3c75277,-0.217193
7,C_ID_f7cada36d3,-0.383835
8,C_ID_9d2bc8dfc4,-0.279446
9,C_ID_6d8dba8475,-0.608058
